<a href="https://colab.research.google.com/github/laxmigurung/APIProject/blob/main/Copy_of_Train_and_Tune_ML_Model_for_Bank_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Necessary Libraries to Load and Explore the Dataset

In [ ]:
pip install pandas numpy scikit-learn xgboost matplotlib opendatasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cudnn-cu12==8.9.2.26; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requ

In [ ]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nicolenavarrete
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling


100%|██████████| 262k/262k [00:00<00:00, 51.8MB/s]

In [ ]:
# reading the CSV file
file =('/content/churn-modelling/Churn_Modelling.csv')
data = pd.read_csv(file)

# displaying the contents of the csv file
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Pre-process the Data
Is the data good to train on?

In [ ]:
# Drop irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# Convert categorical variables to numeric
data = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=1)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


Seperate the data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

# Determine independent and dependent variables
# Features in X (all columns that are used to predict churn as it excludes the target variable) and target variable y (exited indicates that a customer has churned)
X = data.drop('Exited', axis=1)
y = data['Exited']

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

How can we make sure all the column value ranges are on a similar scale?

In [ ]:
from sklearn.preprocessing import StandardScaler

# features with larger ranges can disproportionately influence the model, leading to biased results. Standardization helps to bring all features onto a similar scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Train the Model

What's our model?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss

# Train a basic Logistic Regression model
# Logistic Regression - binary classification tasks(where the goal is to predict a discrete class label)
# Estimates the probabilities that features have a relationship to a binary target variable
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

###Test the Model

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

###Evaluate the Model

In [ ]:
# Evaluate the model performance
# Accuracy - ratio of the number of correct predictions to the total number of predictions, larger number better.
accuracy = accuracy_score(y_test, y_pred)
# Loss function - confidence of model predictions, smaller number indicats that the model's predictions are more closely aligned with the actual outcomes.
logloss = log_loss(y_test, y_prob)
# True Negative, False Positive, False Negative, True Positive
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Initial Model Accuracy: {accuracy * 100.0:.2f}%")
print(f"Initial Model Log-Loss: {logloss:.4f}")
print("Initial Model Confusion Matrix:\n", conf_matrix)

Initial Model Accuracy: 81.10%
Initial Model Log-Loss: 0.4153
Initial Model Confusion Matrix:
 [[1543   64]
 [ 314   79]]


###Tune the Model

How can we adjust how the model trains on the dataset?

In [ ]:
# Tune the model by adjusting the regularization strength (C) and max_iter
# Lower regularization less penalty on loss function (so not memorizing, but understanding)
# Iterations refer to the repeated cycles or steps the algorithm takes to adjust the model's parameters in an effort to minimize a loss function and improve the model's predictions
model_tuned = LogisticRegression(C=0.03, max_iter=300, random_state=42)
model_tuned.fit(X_train, y_train)

LogisticRegression(C=0.03, max_iter=300, random_state=42)

In [ ]:
# Make predictions on the test set
y_pred_tuned = model_tuned.predict(X_test)
y_prob_tuned = model_tuned.predict_proba(X_test)[:, 1]

###Re-Evaluate the Model

In [ ]:
# Evaluate the tuned model performance
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
logloss_tuned = log_loss(y_test, y_prob_tuned)
conf_matrix_tuned = confusion_matrix(y_test, y_pred_tuned)

print(f"Tuned Model Accuracy: {accuracy_tuned * 100.0:.2f}%")
print(f"Tuned Model Log-Loss: {logloss_tuned:.4f}")
print("Tuned Model Confusion Matrix:\n", conf_matrix_tuned)

Tuned Model Accuracy: 81.15%
Tuned Model Log-Loss: 0.4155
Tuned Model Confusion Matrix:
 [[1549   58]
 [ 319   74]]
